# Create Subsequences as Clustering Input
* n-grams
* frequent sequence patterns

In [10]:
import string
from nltk import ngrams

from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.algo.filtering.log.attributes import attributes_filter

In [11]:
filepath = "data/"

log = xes_importer.apply(filepath + 'DomesticDeclarations.xes')

HBox(children=(FloatProgress(value=0.0, description='parsing log, completed traces :: ', max=10500.0, style=ProgressStyle(description_width='initial')), HTML(value='')))

In [12]:
activities = attributes_filter.get_attribute_values(log, "concept:name")
activities

{'Declaration APPROVED by ADMINISTRATION': 8202,
 'Declaration APPROVED by BUDGET OWNER': 2820,
 'Declaration APPROVED by PRE_APPROVER': 685,
 'Declaration FINAL_APPROVED by SUPERVISOR': 10131,
 'Declaration FOR_APPROVAL by ADMINISTRATION': 1,
 'Declaration FOR_APPROVAL by PRE_APPROVER': 1,
 'Declaration FOR_APPROVAL by SUPERVISOR': 1,
 'Declaration REJECTED by ADMINISTRATION': 952,
 'Declaration REJECTED by BUDGET OWNER': 59,
 'Declaration REJECTED by EMPLOYEE': 1365,
 'Declaration REJECTED by MISSING': 91,
 'Declaration REJECTED by PRE_APPROVER': 86,
 'Declaration REJECTED by SUPERVISOR': 293,
 'Declaration SAVED by EMPLOYEE': 135,
 'Declaration SUBMITTED by EMPLOYEE': 11531,
 'Payment Handled': 10044,
 'Request Payment': 10040}

In [13]:
trace_activities = list()

for trace in log:
    event_list = list()
    for event in trace:
        name = event["concept:name"]
        event_list.append(name)
    trace_activities.append(event_list)
trace_activities[:5]

[['Declaration SUBMITTED by EMPLOYEE',
  'Declaration FINAL_APPROVED by SUPERVISOR',
  'Request Payment',
  'Payment Handled'],
 ['Declaration SUBMITTED by EMPLOYEE',
  'Declaration APPROVED by PRE_APPROVER',
  'Declaration FINAL_APPROVED by SUPERVISOR',
  'Request Payment',
  'Payment Handled'],
 ['Declaration SUBMITTED by EMPLOYEE',
  'Declaration APPROVED by PRE_APPROVER',
  'Declaration FINAL_APPROVED by SUPERVISOR',
  'Request Payment',
  'Payment Handled'],
 ['Declaration SUBMITTED by EMPLOYEE',
  'Declaration FINAL_APPROVED by SUPERVISOR',
  'Request Payment',
  'Payment Handled'],
 ['Declaration SUBMITTED by EMPLOYEE',
  'Declaration FINAL_APPROVED by SUPERVISOR',
  'Request Payment',
  'Payment Handled']]

In [14]:
# map activities to single characters
character_dict = dict()
character_list = list(string.ascii_uppercase)

if len(activities) < len(character_list):
    for key, char in zip(activities.keys(), character_list):
            character_dict[key] = char
            
print(character_dict)

{'Declaration FINAL_APPROVED by SUPERVISOR': 'A', 'Declaration APPROVED by ADMINISTRATION': 'K', 'Declaration FOR_APPROVAL by ADMINISTRATION': 'C', 'Declaration SAVED by EMPLOYEE': 'O', 'Declaration APPROVED by PRE_APPROVER': 'E', 'Declaration REJECTED by SUPERVISOR': 'F', 'Declaration SUBMITTED by EMPLOYEE': 'G', 'Request Payment': 'H', 'Declaration REJECTED by EMPLOYEE': 'I', 'Declaration APPROVED by BUDGET OWNER': 'J', 'Declaration FOR_APPROVAL by PRE_APPROVER': 'B', 'Declaration REJECTED by PRE_APPROVER': 'L', 'Declaration REJECTED by ADMINISTRATION': 'M', 'Payment Handled': 'N', 'Declaration FOR_APPROVAL by SUPERVISOR': 'D', 'Declaration REJECTED by BUDGET OWNER': 'P', 'Declaration REJECTED by MISSING': 'Q'}


In [15]:
# convert list of all sentences and map it to characters
char_sentences = list()

for trace in trace_activities:
    char_sentence = list()
    for activity in trace:
        #print(activity, character_dict[activity])
        char_sentence.append(character_dict[activity])
    char_sentences.append(char_sentence)
    
print(char_sentences)

[['G', 'A', 'H', 'N'], ['G', 'E', 'A', 'H', 'N'], ['G', 'E', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'Q', 'G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'Q', 'G', 'L', 'I', 'G', 'E', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'Q', 'G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['O'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'Q', 'G', 'A', 'H', 'N'], ['G', 'E', 'A', 'H', 'N'], ['G', 'A', 'Q', 'G', 'A', 'H', 'N'], ['G', 'A', 'Q', 'G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'E', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G', 'A', 'H', 'N'], ['G'

## N-grams


In [16]:
n = 3
sixgrams = ngrams(" ".join(char_sentences[0]).split(), n)
ngrams_list = list()

for grams in sixgrams:
    print(grams)
    ngrams_list.append(grams)

('G', 'A', 'H')
('A', 'H', 'N')


## Frequent Sequence Patterns

# Clustering

In [17]:
import numpy as np
import scipy.cluster.hierarchy
#from pyxdameraulevenshtein import damerau_levenshtein_distance
from collections import defaultdict

In [18]:
def cluster_ngrams(ngrams, compute_distance, max_dist, method):
    """
    Cluster ngrams.
    Params:
        ngrams: [list] List of tuple of words in each ngram to cluster.
        compute_distance: [func] Function that computes distance between two
            pairs of ngrams.
        max_dist: [float] Maximum distance allowed for two clusters to merge.
        method: [string] Method to use for clustering.  'single',
            'complete', 'average', 'centroid', 'median', 'ward', or 'weighted'.
            See http://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html for details.
    Returns:
        clusters: [list] List of ngrams in each cluster.
    """
    indices = np.triu_indices(len(ngrams), 1)
    pairwise_dists = np.apply_along_axis(
        lambda col: compute_distance(ngrams[col[0]], ngrams[col[1]]),
        0, indices)
    hierarchy = scipy.cluster.hierarchy.linkage(pairwise_dists, method=method)
    clusters = dict((i, [i]) for i in range(len(ngrams)))
    for (i, iteration) in enumerate(hierarchy):
        cl1, cl2, dist, num_items = iteration
        if dist >  max_dist:
            break
        items1 = clusters[cl1]
        items2 = clusters[cl2]
        del clusters[cl1]
        del clusters[cl2]
        clusters[len(ngrams) + i] = items1 + items2
    ngram_clusters = []
    for cluster in clusters.values():
        ngram_clusters.append([ngrams[i] for i in cluster])
    return ngram_clusters

def levenshtein(seq1, seq2):
    size_x = len(seq1) + 1
    size_y = len(seq2) + 1
    matrix = np.zeros ((size_x, size_y))
    for x in range(size_x):
        matrix [x, 0] = x
    for y in range(size_y):
        matrix [0, y] = y

    for x in range(1, size_x):
        for y in range(1, size_y):
            if seq1[x-1] == seq2[y-1]:
                matrix [x,y] = min(
                    matrix[x-1, y] + 1,
                    matrix[x-1, y-1],
                    matrix[x, y-1] + 1
                )
            else:
                matrix [x,y] = min(
                    matrix[x-1,y] + 1,
                    matrix[x-1,y-1] + 1,
                    matrix[x,y-1] + 1
                )
    print (matrix)
    return (matrix[size_x - 1, size_y - 1])


def dl_ngram_dist(ngram1, ngram2):
    """
    Compute distance between ngrams by summing the Damerau-Levenshtein distance
    for consecutive words in ngrams.
    Params:
        ngram1: [tuple] Tuple of words.
        ngram2: [tuple] Tuple of words.
    Returns:
        distance [int] Measure of distance between two ngrams.
    """
    return sum(levenshtein(w1, w2) for w1, w2 in zip(ngram1,
        ngram2))

In [20]:
cluster_ngrams(ngrams_list, dl_ngram_dist, 3, 'single')

[[0. 1.]
 [1. 1.]]
[[0. 1.]
 [1. 1.]]
[[0. 1.]
 [1. 1.]]


[[('G', 'A', 'H'), ('A', 'H', 'N')]]

In [21]:
ngrams_list


[('G', 'A', 'H'), ('A', 'H', 'N')]

In [22]:
ngrams = [
            ['from', 'my', 'house'],
            ['from', 'my', 'hose'],
            ['he', 'was', 'eating'],
            ['she', 'was', 'eating'],
            ['fell', 'asleep', 'on'],
            ['moved', 'to', 'a'],
            ['rom', 'my', 'house'],
            ['from', 'my', 'house'],
        ]